In [127]:
#################################################################
#Developed by Jonathan Ojeda 20-01-2021 QAAFI-UQ
#Functionality:
#Convert from date to julian day
#Get soybean planting dates by US state using data from NASS-USDA
#Convert dtypes
#Locate specific rows and make a new dataframe
#create new rows with data
#Create campaig file for planting date
#Use of pivot table to trnspose data
#################################################################

In [1]:
import os
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import dateutil
import pylab as py
import seaborn as sns
import scipy 
from scipy import stats
import sklearn.metrics
from numpy  import array
import glob
import functools
from functools import reduce
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)
import statsmodels.api as sm
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<ipython-input-1-9e9d1d5ccf14>:19: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)


In [55]:
#read data
df=pd.read_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\USDA\PlantedDateCountySorghum.csv')

#drop columns
df.drop(['freq_desc', 'county_name', 'util_practice_desc',
       'country_code', 'watershed_desc', 'domain_desc', 'class_desc',
       'county_code', 'region_desc', 'begin_code', 'load_time', 'county_ansi',
       'short_desc', 'source_desc', 'group_desc',
       'unit_desc', 'CV (%)', 'reference_period_desc', 'zip_5',
       'asd_desc', 'congr_district_code', 'location_desc',
       'prodn_practice_desc', 'domaincat_desc', 'commodity_desc',
       'country_name', 'watershed_code', 'sector_desc',
       'state_fips_code', 'agg_level_desc', 'statisticcat_desc', 'asd_code',
       'end_code'], axis=1, inplace=True)

#convert dtype
df['week_ending'] = df['week_ending'].astype('datetime64[ns]')

#convert from date to julian day 
df['newFormat'] = df['week_ending'].dt.strftime('%y%j')
df['jday'] = df['newFormat'].str.slice(2,5)

#locate specific rows and make a new dataframe
df.loc[(df['Value'] > 50) & (df['Value'] < 100), 'pdate'] = df['jday']

#drop nan values
df2 = df.dropna()

df2

,state_alpha,week_ending,year,Value,state_name,state_ansi,newFormat,jday,pdate
2,CO,1979-05-20,1979,69,COLORADO,8,79140,140,140
3,CO,1979-05-27,1979,87,COLORADO,8,79147,147,147
4,CO,1979-03-06,1979,92,COLORADO,8,79065,065,065
5,GA,1979-04-22,1979,91,GEORGIA,13,79112,112,112
6,GA,1979-04-29,1979,94,GEORGIA,13,79119,119,119
...,...,...,...,...,...,...,...,...,...
49404,WI,2020-04-10,2020,90,WISCONSIN,55,20101,101,101
49405,WI,2020-11-10,2020,96,WISCONSIN,55,20315,315,315
49412,WY,2020-09-27,2020,77,WYOMING,56,20271,271,271
49413,WY,2020-04-10,2020,96,WYOMING,56,20101,101,101


In [56]:
#convert dtype
df2['pdate'] = df2['pdate'].astype('int64')

#make a mean across years and remove states
pdate = df2.groupby(['state_alpha', 'state_ansi', 'state_name'],as_index=False).mean().round(decimals=0)
pdate.drop(['year'], axis=1, inplace=True)
pdate = pdate[pdate.state_name != 'IDAHO']
pdate = pdate[pdate.state_name != 'NORTH DAKOTA']
pdate = pdate[pdate.state_name != 'SOUTH DAKOTA']
pdate = pdate[pdate.state_name != 'WEST VIRGINIA']

#sort dataframe by column
pdate4 = pdate.sort_values(by=['pdate'])
pdate4
#pdate4.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\Maps\SorghumMask\comp.csv')

<ipython-input-56-4b569ad0c46e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pdate'] = df2['pdate'].astype('int64')


,state_alpha,state_ansi,state_name,Value,pdate
34,SC,45,SOUTH CAROLINA,85.0,181.0
12,LA,22,LOUISIANA,86.0,181.0
0,AL,1,ALABAMA,82.0,188.0
27,NV,32,NEVADA,69.0,190.0
19,MS,28,MISSISSIPPI,85.0,190.0
1,AR,5,ARKANSAS,85.0,193.0
3,CT,9,CONNECTICUT,81.0,195.0
5,GA,13,GEORGIA,91.0,196.0
36,TN,47,TENNESSEE,86.0,197.0
13,MA,25,MASSACHUSETTS,81.0,199.0


In [57]:
#create new rows with data
pdate2 = pdate.append(pd.DataFrame([['DC',51,'DISTRICT OF COLUMBIA',55,205]], columns=pdate.columns))
pdate3 = pdate2.append(pd.DataFrame([['FL',13,'FLORIDA',55,182]], columns=pdate2.columns))

In [31]:
pdate3.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\inputs\pdateCorn.csv')

In [58]:
pdate3.rename(columns={'state_ansi':'STATE_FIPS'}, inplace=True)

In [62]:
pdate3.pdate.describe()

count     43.000000
mean     205.279070
std       10.583738
min      181.000000
25%      199.500000
50%      209.000000
75%      213.000000
max      222.000000
Name: pdate, dtype: float64

## Create campaign file for psims

In [90]:
#read data
latlon = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/LatLonByState.csv')
pdate = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/pdates.csv')
tile = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/Grids40KmList.csv')
camp = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/CampaignLatLonList.csv')

In [91]:
#work only with jday50, lat and lon, columns
psimspre = pd.merge(latlon, pdate3, how="outer", on="STATE_FIPS")
psims2 = psimspre.dropna()
final = pd.merge(camp, psims2, how="outer", on=["lat","lon"])
final2 = final.drop(['state', 'STATE_FIPS'],axis=1)
final2[['pdate']] = final2[['pdate']].fillna(axis=1, value=218)
final2[['pdate']] = final2[['pdate']].round(decimals=0)

In [96]:
final2.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\inputs\ss.csv')

In [94]:
#Use pivot table to create the array for the campaign file in psims
final3 = final2.pivot_table(values='pdate',index='lat',columns='lon', dropna=False)
final3.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\inputs\pdateCampaign.csv')